# **Imputation**

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

In [3]:
chem = pd.read_csv('ChemicalProcess.csv')
chem.head()

,Yield,BiologicalMaterial01,BiologicalMaterial02,BiologicalMaterial03,BiologicalMaterial04,BiologicalMaterial05,BiologicalMaterial06,BiologicalMaterial07,BiologicalMaterial08,BiologicalMaterial09,BiologicalMaterial10,BiologicalMaterial11,BiologicalMaterial12,ManufacturingProcess01,ManufacturingProcess02,ManufacturingProcess03,ManufacturingProcess04,ManufacturingProcess05,ManufacturingProcess06,ManufacturingProcess07,ManufacturingProcess08,ManufacturingProcess09,ManufacturingProcess10,ManufacturingProcess11,ManufacturingProcess12,ManufacturingProcess13,ManufacturingProcess14,ManufacturingProcess15,ManufacturingProcess16,ManufacturingProcess17,ManufacturingProcess18,ManufacturingProcess19,ManufacturingProcess20,ManufacturingProcess21,ManufacturingProcess22,ManufacturingProcess23,ManufacturingProcess24,ManufacturingProcess25,ManufacturingProcess26,ManufacturingProcess27,ManufacturingProcess28,ManufacturingProcess29,ManufacturingProcess30,ManufacturingProcess31,ManufacturingProcess32,ManufacturingProcess33,ManufacturingProcess34,ManufacturingProcess35,ManufacturingProcess36,ManufacturingProcess37,ManufacturingProcess38,ManufacturingProcess39,ManufacturingProcess40,ManufacturingProcess41,ManufacturingProcess42,ManufacturingProcess43,ManufacturingProcess44,ManufacturingProcess45
0,38.00,6.25,49.58,56.97,12.74,19.51,43.73,100.0,16.66,11.44,3.46,138.09,18.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.00,NaN,NaN,NaN,35.5,4898.0,6108,4682,35.5,4865,6049,4665,0.0,NaN,NaN,NaN,4873.0,6074.0,4685.0,10.7,21.0,9.9,69.1,156,66.0,2.4,486.0,0.019,0.5,3,7.2,NaN,NaN,11.6,3.0,1.8,2.4
1,42.44,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.0,0.0,NaN,917.0,1032.2,210.0,177.0,178.0,46.57,NaN,NaN,0.0,34.0,4869.0,6095,4617,34.0,4867,6097,4621,0.0,3.0,0.0,3.0,4869.0,6107.0,4630.0,11.2,21.4,9.9,68.7,169,66.0,2.6,508.0,0.019,2.0,2,7.2,0.1,0.15,11.1,0.9,1.9,2.2
2,42.03,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.0,0.0,NaN,912.0,1003.6,207.1,178.0,178.0,45.07,NaN,NaN,0.0,34.8,4878.0,6087,4617,34.8,4877,6078,4621,0.0,4.0,1.0,4.0,4897.0,6116.0,4637.0,11.1,21.3,9.4,69.3,173,66.0,2.6,509.0,0.018,0.7,2,7.2,0.0,0.00,12.0,1.0,1.8,2.3
3,41.42,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.0,0.0,NaN,911.0,1014.6,213.3,177.0,177.0,44.92,NaN,NaN,0.0,34.8,4897.0,6102,4635,34.8,4872,6073,4611,0.0,5.0,2.0,5.0,4892.0,6111.0,4630.0,11.1,21.3,9.4,69.3,171,68.0,2.5,496.0,0.018,1.2,2,7.2,0.0,0.00,10.6,1.1,1.8,2.1
4,42.49,7.47,63.33,72.25,14.02,17.91,54.66,100.0,18.22,12.80,3.05,147.61,21.05,10.7,0.0,NaN,918.0,1027.5,205.7,178.0,178.0,44.96,NaN,NaN,0.0,34.6,4992.0,6233,4733,33.9,4886,6102,4659,-0.7,8.0,4.0,18.0,4930.0,6151.0,4684.0,11.3,21.6,9.0,69.4,171,70.0,2.5,468.0,0.017,0.2,2,7.3,0.0,0.00,11.0,1.1,1.7,2.1


In [5]:
print("Number of NaN values in the dataframe: ", chem.isnull().sum().sum())

Number of NaN values in the dataframe:  106


## **Constant Strategy**

In [9]:
from sklearn.impute import SimpleImputer

imp_const = SimpleImputer(strategy='constant', fill_value=0.5).set_output(transform='pandas')

chem_const = imp_const.fit_transform(chem)

chem_const.isnull().sum().sum()

In [12]:
chem_const.head(3)

,Yield,BiologicalMaterial01,BiologicalMaterial02,BiologicalMaterial03,BiologicalMaterial04,BiologicalMaterial05,BiologicalMaterial06,BiologicalMaterial07,BiologicalMaterial08,BiologicalMaterial09,BiologicalMaterial10,BiologicalMaterial11,BiologicalMaterial12,ManufacturingProcess01,ManufacturingProcess02,ManufacturingProcess03,ManufacturingProcess04,ManufacturingProcess05,ManufacturingProcess06,ManufacturingProcess07,ManufacturingProcess08,ManufacturingProcess09,ManufacturingProcess10,ManufacturingProcess11,ManufacturingProcess12,ManufacturingProcess13,ManufacturingProcess14,ManufacturingProcess15,ManufacturingProcess16,ManufacturingProcess17,ManufacturingProcess18,ManufacturingProcess19,ManufacturingProcess20,ManufacturingProcess21,ManufacturingProcess22,ManufacturingProcess23,ManufacturingProcess24,ManufacturingProcess25,ManufacturingProcess26,ManufacturingProcess27,ManufacturingProcess28,ManufacturingProcess29,ManufacturingProcess30,ManufacturingProcess31,ManufacturingProcess32,ManufacturingProcess33,ManufacturingProcess34,ManufacturingProcess35,ManufacturingProcess36,ManufacturingProcess37,ManufacturingProcess38,ManufacturingProcess39,ManufacturingProcess40,ManufacturingProcess41,ManufacturingProcess42,ManufacturingProcess43,ManufacturingProcess44,ManufacturingProcess45
0,38.00,6.25,49.58,56.97,12.74,19.51,43.73,100.0,16.66,11.44,3.46,138.09,18.83,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,43.00,0.5,0.5,0.5,35.5,4898.0,6108.0,4682.0,35.5,4865.0,6049.0,4665.0,0.0,0.5,0.5,0.5,4873.0,6074.0,4685.0,10.7,21.0,9.9,69.1,156.0,66.0,2.4,486.0,0.019,0.5,3.0,7.2,0.5,0.50,11.6,3.0,1.8,2.4
1,42.44,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.0,0.0,0.5,917.0,1032.2,210.0,177.0,178.0,46.57,0.5,0.5,0.0,34.0,4869.0,6095.0,4617.0,34.0,4867.0,6097.0,4621.0,0.0,3.0,0.0,3.0,4869.0,6107.0,4630.0,11.2,21.4,9.9,68.7,169.0,66.0,2.6,508.0,0.019,2.0,2.0,7.2,0.1,0.15,11.1,0.9,1.9,2.2
2,42.03,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.0,0.0,0.5,912.0,1003.6,207.1,178.0,178.0,45.07,0.5,0.5,0.0,34.8,4878.0,6087.0,4617.0,34.8,4877.0,6078.0,4621.0,0.0,4.0,1.0,4.0,4897.0,6116.0,4637.0,11.1,21.3,9.4,69.3,173.0,66.0,2.6,509.0,0.018,0.7,2.0,7.2,0.0,0.00,12.0,1.0,1.8,2.3


## **Mean Strategy**

In [14]:
imp_mean = SimpleImputer(strategy = 'mean').set_output(transform='pandas')

chem_mean = imp_mean.fit_transform(chem)

print("Number of NaN values in the 'chem_mean' dataframe: ", chem_mean.isnull().sum().sum())

chem_mean.head(3)

Number of NaN values in the 'chem_mean' dataframe:  0


,Yield,BiologicalMaterial01,BiologicalMaterial02,BiologicalMaterial03,BiologicalMaterial04,BiologicalMaterial05,BiologicalMaterial06,BiologicalMaterial07,BiologicalMaterial08,BiologicalMaterial09,BiologicalMaterial10,BiologicalMaterial11,BiologicalMaterial12,ManufacturingProcess01,ManufacturingProcess02,ManufacturingProcess03,ManufacturingProcess04,ManufacturingProcess05,ManufacturingProcess06,ManufacturingProcess07,ManufacturingProcess08,ManufacturingProcess09,ManufacturingProcess10,ManufacturingProcess11,ManufacturingProcess12,ManufacturingProcess13,ManufacturingProcess14,ManufacturingProcess15,ManufacturingProcess16,ManufacturingProcess17,ManufacturingProcess18,ManufacturingProcess19,ManufacturingProcess20,ManufacturingProcess21,ManufacturingProcess22,ManufacturingProcess23,ManufacturingProcess24,ManufacturingProcess25,ManufacturingProcess26,ManufacturingProcess27,ManufacturingProcess28,ManufacturingProcess29,ManufacturingProcess30,ManufacturingProcess31,ManufacturingProcess32,ManufacturingProcess33,ManufacturingProcess34,ManufacturingProcess35,ManufacturingProcess36,ManufacturingProcess37,ManufacturingProcess38,ManufacturingProcess39,ManufacturingProcess40,ManufacturingProcess41,ManufacturingProcess42,ManufacturingProcess43,ManufacturingProcess44,ManufacturingProcess45
0,38.00,6.25,49.58,56.97,12.74,19.51,43.73,100.0,16.66,11.44,3.46,138.09,18.83,11.207429,16.682659,1.539565,931.851429,1001.693143,207.401724,177.48,177.554286,43.00,9.179042,9.385542,857.811429,35.5,4898.0,6108.0,4682.0,35.5,4865.0,6049.0,4665.0,0.0,5.405714,3.017143,8.834286,4873.0,6074.0,4685.0,10.7,21.0,9.9,69.1,156.0,66.0,2.4,486.0,0.019,0.5,3.0,7.2,0.017714,0.023714,11.6,3.0,1.8,2.4
1,42.44,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.000000,0.000000,1.539565,917.000000,1032.200000,210.000000,177.00,178.000000,46.57,9.179042,9.385542,0.000000,34.0,4869.0,6095.0,4617.0,34.0,4867.0,6097.0,4621.0,0.0,3.000000,0.000000,3.000000,4869.0,6107.0,4630.0,11.2,21.4,9.9,68.7,169.0,66.0,2.6,508.0,0.019,2.0,2.0,7.2,0.100000,0.150000,11.1,0.9,1.9,2.2
2,42.03,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.000000,0.000000,1.539565,912.000000,1003.600000,207.100000,178.00,178.000000,45.07,9.179042,9.385542,0.000000,34.8,4878.0,6087.0,4617.0,34.8,4877.0,6078.0,4621.0,0.0,4.000000,1.000000,4.000000,4897.0,6116.0,4637.0,11.1,21.3,9.4,69.3,173.0,66.0,2.6,509.0,0.018,0.7,2.0,7.2,0.000000,0.000000,12.0,1.0,1.8,2.3


## **Median Strategy**

In [16]:
imp_median = SimpleImputer(strategy = 'median').set_output(transform='pandas')

chem_median = imp_median.fit_transform(chem)

print("Number of NaN values in the 'chem_median' dataframe: ", chem_median.isnull().sum().sum())

chem_median.head(3)

Number of NaN values in the 'chem_median' dataframe:  0


,Yield,BiologicalMaterial01,BiologicalMaterial02,BiologicalMaterial03,BiologicalMaterial04,BiologicalMaterial05,BiologicalMaterial06,BiologicalMaterial07,BiologicalMaterial08,BiologicalMaterial09,BiologicalMaterial10,BiologicalMaterial11,BiologicalMaterial12,ManufacturingProcess01,ManufacturingProcess02,ManufacturingProcess03,ManufacturingProcess04,ManufacturingProcess05,ManufacturingProcess06,ManufacturingProcess07,ManufacturingProcess08,ManufacturingProcess09,ManufacturingProcess10,ManufacturingProcess11,ManufacturingProcess12,ManufacturingProcess13,ManufacturingProcess14,ManufacturingProcess15,ManufacturingProcess16,ManufacturingProcess17,ManufacturingProcess18,ManufacturingProcess19,ManufacturingProcess20,ManufacturingProcess21,ManufacturingProcess22,ManufacturingProcess23,ManufacturingProcess24,ManufacturingProcess25,ManufacturingProcess26,ManufacturingProcess27,ManufacturingProcess28,ManufacturingProcess29,ManufacturingProcess30,ManufacturingProcess31,ManufacturingProcess32,ManufacturingProcess33,ManufacturingProcess34,ManufacturingProcess35,ManufacturingProcess36,ManufacturingProcess37,ManufacturingProcess38,ManufacturingProcess39,ManufacturingProcess40,ManufacturingProcess41,ManufacturingProcess42,ManufacturingProcess43,ManufacturingProcess44,ManufacturingProcess45
0,38.00,6.25,49.58,56.97,12.74,19.51,43.73,100.0,16.66,11.44,3.46,138.09,18.83,11.4,21.0,1.54,934.0,999.2,206.8,177.0,178.0,43.00,9.1,9.4,0.0,35.5,4898.0,6108.0,4682.0,35.5,4865.0,6049.0,4665.0,0.0,5.0,3.0,8.0,4873.0,6074.0,4685.0,10.7,21.0,9.9,69.1,156.0,66.0,2.4,486.0,0.019,0.5,3.0,7.2,0.0,0.00,11.6,3.0,1.8,2.4
1,42.44,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.0,0.0,1.54,917.0,1032.2,210.0,177.0,178.0,46.57,9.1,9.4,0.0,34.0,4869.0,6095.0,4617.0,34.0,4867.0,6097.0,4621.0,0.0,3.0,0.0,3.0,4869.0,6107.0,4630.0,11.2,21.4,9.9,68.7,169.0,66.0,2.6,508.0,0.019,2.0,2.0,7.2,0.1,0.15,11.1,0.9,1.9,2.2
2,42.03,8.01,60.97,67.48,14.65,19.36,53.14,100.0,19.04,12.55,3.46,153.67,21.05,0.0,0.0,1.54,912.0,1003.6,207.1,178.0,178.0,45.07,9.1,9.4,0.0,34.8,4878.0,6087.0,4617.0,34.8,4877.0,6078.0,4621.0,0.0,4.0,1.0,4.0,4897.0,6116.0,4637.0,11.1,21.3,9.4,69.3,173.0,66.0,2.6,509.0,0.018,0.7,2.0,7.2,0.0,0.00,12.0,1.0,1.8,2.3


## Applying Strategy : mean median

In [22]:
impmean=SimpleImputer(strategy="mean").set_output(transform="pandas")
chemnew=impmean.fit_transform(chem)
y=chemnew['Yield']
X=chemnew.drop('Yield',axis=1)

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=24)

In [24]:
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
r2_score(y_test,y_pred)

0.2418100595510433

In [25]:
impmean=SimpleImputer(strategy="median").set_output(transform="pandas")
chemnew=impmean.fit_transform(chem)
y=chemnew['Yield']
X=chemnew.drop('Yield',axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=24)
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
r2_score(y_test,y_pred)

0.2366120304066618

In [26]:
boston=pd.read_csv('Boston.csv')
X=boston.drop("medv",axis=1)
y=boston["medv"]

# **K-Folds and Cross-Validation**

In [30]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

In [31]:
boston = pd.read_csv('Boston.csv')

X = boston.drop("medv", axis=1)
y = boston["medv"]

**Linear Regression**

In [42]:
lr = LinearRegression()
result = cross_val_score(lr, X, y)
print("Scores Array for K-folds: ",result)

print("Mean scores: ", result.mean())

Scores Array for K-folds:  [ 0.63919994  0.71386698  0.58702344  0.07923081 -0.25294154]
Mean scores:  0.3532759243958849


**Elastic Net**

In [44]:
elastic = ElasticNet()
result = cross_val_score(elastic, X, y)
print("Scores Array for K-folds: ", result)
print("Mean scores: ", result.mean())

Scores Array for K-folds:  [0.57022044 0.6626767  0.40322405 0.45880379 0.26833761]
Mean scores:  0.4726525191941059


### **Hyper Parameter Optimization with Cross-Validation-Score for Elastic Net**

In [53]:
alphas = np.linspace(0.0001, 10, 20)
l1     = np.linspace(0.0001, 1,  10)

scores = []

for a in alphas:
  for i in l1:
    elastic = ElasticNet(alpha = a, l1_ratio = i)
    scores.append( [ a, i, cross_val_score(elastic, X, y).mean() ] )


df_scores = pd.DataFrame(scores, columns = ['alpha', 'l1_ratio', 'score'])

df_scores.sort_values('score', ascending = False, inplace = True)

Best_alpha = df_scores['alpha'].iloc[0]
Best_l1 = df_scores['l1_ratio'].iloc[0]
Best_score = df_scores['score'].iloc[0]

print("Best alpha: ", Best_alpha)
print("Best l1_ratio: ", Best_l1)
print("Best score: ", Best_score)
print("\n")
df_scores.head()

Best alpha:  0.5264105263157894
Best l1_ratio:  0.0001
Best score:  0.4976889030997101




,alpha,l1_ratio,score
10,0.526411,0.0001,0.497689
11,0.526411,0.1112,0.495480
12,0.526411,0.2223,0.492985
13,0.526411,0.3334,0.490010
20,1.052721,0.0001,0.487077


**Sophisticated Way to do Hyper-Parameter Tuning**

Using `GridSerachCV()`

In [54]:
from sklearn.model_selection import GridSearchCV

elastic = ElasticNet()

params = {'alpha': np.linspace(0.0001, 10, 20), 'l1_ratio': np.linspace(0.0001, 1, 10)}

gcv_el = GridSearchCV(elastic, param_grid = params)

gcv_el.fit(X, y)  #This statement { .fit() } is running the loop internally

GridSearchCV(estimator=ElasticNet(),
             param_grid={'alpha': array([1.00000000e-04, 5.26410526e-01, 1.05272105e+00, 1.57903158e+00,
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00])})

In [55]:
print("Best Params: ", gcv_el.best_params_)
print("Best Score: ", gcv_el.best_score_)

Best Params:  {'alpha': 0.5264105263157894, 'l1_ratio': 0.0001}
Best Score:  0.4976889030997101


## Using object of pipeline inside GridSearchCV()

In [56]:
chem=pd.read_csv("ChemicalProcess.csv")
y=chem["Yield"]
X=chem.drop("Yield",axis=1)


In [57]:
imp_mean=SimpleImputer(strategy="mean")
lr=LinearRegression()
pipe_lr=Pipeline([("IMP",imp_mean),("LR",lr)])
res_lr=cross_val_score(pipe_lr,X,y).mean()
res_lr

-94.57748595778229

In [58]:
imp_median=SimpleImputer(strategy="median")
lr=LinearRegression()
pipe_lr=Pipeline([("IMP",imp_median),("LR",lr)])
res_lr=cross_val_score(pipe_lr,X,y).mean()
res_lr

-86.99645845799377

In [60]:
imp=SimpleImputer()
pipe_lr=Pipeline([("IMP",imp),("LR",lr)])
params={"IMP__strategy":["mean","median"]}
gcv_lr=GridSearchCV(pipe_lr,param_grid=params)
gcv_lr.fit(X,y)
print("Best Params: ",gcv_lr.best_params_)
print("Best Score: ",gcv_lr.best_score_)

Best Params:  {'IMP__strategy': 'median'}
Best Score:  -86.99645845799377


### **Use Pipe + GridSearchCV + Imputer + ElasticNet**

In [ ]:
pipe_lr.get_params()

In [66]:
imp = SimpleImputer()
pipe_el = Pipeline( [ ("IMP", imp), ("EL", elastic) ] )
params = {
    "IMP__strategy" : ["mean", "median"],
    "EL__alpha" : np.linspace(0.0001, 10, 20),
    "EL__l1_ratio" : np.linspace(0.0001, 1, 10)
    }

gcv_el = GridSearchCV(pipe_el, param_grid = params)
gcv_el.fit(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.126e+01, tolerance: 4.218e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.350e+01, tolerance: 4.172e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.983e+01, tolerance: 5.288e

GridSearchCV(estimator=Pipeline(steps=[('IMP', SimpleImputer()),
                                       ('EL', ElasticNet())]),
             param_grid={'EL__alpha': array([1.00000000e-04, 5.26410526e-01, 1.05272105e+00, 1.57903158e+00,
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'EL__l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00]),
                         'IMP__strategy': ['mean', 'median']})

**Export the grid results to CSV**
`def_results.to_csv('path')`


In [67]:
print("Best Params: ", gcv_el.best_params_)
print("Best Score: ", gcv_el.best_score_)

Best Params:  {'EL__alpha': 10.0, 'EL__l1_ratio': 1.0, 'IMP__strategy': 'median'}
Best Score:  -1.5517262087776742


# **Housing Dataset**

In [68]:
housing=pd.read_csv('Housing.csv')
housing.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [86]:
y = housing['price']
X = housing.drop('price', axis = 1)

In [87]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [88]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [89]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output = False, drop = 'first')

In [96]:
ct = make_column_transformer( ('passthrough', make_column_selector(dtype_exclude = object)),
                             (ohe, make_column_selector(dtype_include = object)),
                              verbose_feature_names_out=False).set_output(transform = 'pandas')

elastic = ElasticNet()

pipe_el = Pipeline( [ ("CT", ct), ("EL", elastic) ] )

params = { "EL__alpha": np.linspace(0.0001, 10, 20),
            "EL__l1_ratio": np.linspace(0.0001, 1, 10)
        }

gc = GridSearchCV(pipe_el, param_grid = params,verbose=3)
gc.fit(X, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 1/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.441 total time=   0.0s
[CV 2/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.502 total time=   0.0s
[CV 3/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.463 total time=   0.0s
[CV 4/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.519 total time=   0.0s
[CV 5/5] END EL__alpha=0.0001, EL__l1_ratio=0.0001;, score=0.617 total time=   0.0s
[CV 1/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.441 total time=   0.0s
[CV 2/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.502 total time=   0.0s
[CV 3/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.463 total time=   0.0s
[CV 4/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.519 total time=   0.0s
[CV 5/5] END EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001;, score=0.617 total time=   0.0s
[CV 1/5] END EL__alpha=0.0001, 

GridSearchCV(estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7e944006f4c0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7e943be89510>)],
                                                          verbose_featur...
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'EL__l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00])},
             verbose=3)

In [91]:
print("Best Params: ", gc.best_params_)
print("Best Score: ", gc.best_score_)

Best Params:  {'EL__alpha': 10.0, 'EL__l1_ratio': 1.0}
Best Score:  0.5084207009081491


# **LOGISTIC REGRESSION**

# **Breast Cancer Wisconsin (ORIGINAL)**

In [ ]:
# pip install ucimlrepo

In [122]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score

# fetch dataset
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_original.data.features
y = breast_cancer_wisconsin_original.data.targets

In [123]:
# X.isnull().sum()

X.drop("Bare_nuclei",axis=1,inplace=True)
X

<ipython-input-123-184a86b05d9c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop("Bare_nuclei",axis=1,inplace=True)


,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bland_chromatin,Normal_nucleoli,Mitoses
0,5,1,1,1,2,3,1,1
1,5,4,4,5,7,3,2,1
2,3,1,1,1,2,3,1,1
3,6,8,8,1,3,3,7,1
4,4,1,1,3,2,3,1,1
...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,1,1,1
695,2,1,1,1,2,1,1,1
696,5,10,10,3,7,8,10,2
697,4,8,6,4,3,10,6,1


In [124]:
X.isnull().sum()

,0
Clump_thickness,0
Uniformity_of_cell_size,0
Uniformity_of_cell_shape,0
Marginal_adhesion,0
Single_epithelial_cell_size,0
Bland_chromatin,0
Normal_nucleoli,0
Mitoses,0


In [125]:
y["Class"]=np.where(y["Class"]==2,0,1)
y["Class"].value_counts()

<ipython-input-125-6f1f6230abbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["Class"]=np.where(y["Class"]==2,0,1)


,count
Class,
0,458
1,241


In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=24)

In [133]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print(confusion_matrix(y_test["Class"].values,y_pred))
print(accuracy_score(y_test["Class"].values,y_pred))

[[135   3]
 [  4  68]]
0.9666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [128]:
# pd.crosstab(y_test["Class"],y_pred)

In [169]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

In [172]:
y['Diagnosis'].unique()

array(['M', 'B'], dtype=object)

In [174]:
y["Diagnosis"] = np.where(y["Diagnosis"] == "B", 0, 1)

<ipython-input-174-ed870e4a0a27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["Diagnosis"] = np.where(y["Diagnosis"] == "B", 0, 1)


In [175]:
y['Diagnosis'].unique()

array([1, 0])

**Naive Rule/Baseline Model** ⚓

In [176]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [177]:
X.shape

(569, 30)

In [178]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = np.zeros(y_test.shape[0])

print(confusion_matrix(y_test["Diagnosis"], y_pred))

y_pred = lr.predict(X_test)

print(confusion_matrix(y_test["Diagnosis"], y_pred))

print(accuracy_score(y_test,y_pred))

print("Accuracy Score: ", accuracy_score(y_test,y_pred))

In [164]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [165]:
y_pred = lr.predict(X_test)

print(confusion_matrix(y_test["Diagnosis"], y_pred))

print(accuracy_score(y_test["Diagnosis"], y_pred))

[[100   6]
 [  5  60]]
0.935672514619883


# **IRIS Dataset**

In [35]:
from sklearn.preprocessing import LabelEncoder

In [37]:
iris = pd.read_csv('iris.csv')
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [38]:
iris.columns

Index(['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width',
       'Species'],
      dtype='object')

In [39]:
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [40]:
iris.isnull().sum()

,0
Sepal.Length,0
Sepal.Width,0
Petal.Length,0
Petal.Width,0
Species,0


In [41]:
y = iris['Species']
X = iris.drop('Species', axis = 1)

In [42]:
lbl = LabelEncoder()
y = lbl.fit_transform(y)
y.max()

2

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)


In [44]:
y_pred=np.ones(y_test.shape[0])*y.max()
y_pred

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [45]:
print(confusion_matrix(y_test,y_pred))
y_test.shape

[[ 0  0 15]
 [ 0  0 12]
 [ 0  0 18]]


(45,)

In [46]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [47]:
y_pred = lr.predict(X_test)

In [48]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[15  0  0]
 [ 0 11  1]
 [ 0  0 18]]


In [49]:
accuracy_score(y_test, y_pred)

0.9777777777777777

# **Default.csv**

In [26]:
# Label encoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# train_test_split
from sklearn.model_selection import train_test_split
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# Confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score

In [27]:
default=pd.read_csv("Default.csv")
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.62507
1,No,Yes,817.180407,12106.13470
2,No,No,1073.549164,31767.13895
3,No,No,529.250605,35704.49394
4,No,No,785.655883,38463.49588


In [28]:
le=LabelEncoder()
default["default"]=le.fit_transform(default["default"])
he=OneHotEncoder(sparse_output=False,drop="first")
default["student"]=he.fit_transform(default[["student"]])

In [29]:

default["default"].value_counts()

,count
default,
0,9667
1,333


In [34]:
y = default["default"]
X = default.drop("default",axis=1)

In [31]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3,random_state=24)

In [33]:
y_p = np.zeros(y_test.shape[0])
print("Confusion Matrix:\n", confusion_matrix(y_test, y_p))
print("Accuracy Score", accuracy_score(y_test, y_p))

Confusion Matrix:
 [[2901    0]
 [  99    0]]
Accuracy Score 0.967


In [20]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [22]:
y_pred = lr.predict(X_test)

In [24]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score", accuracy_score(y_test, y_pred))

Confusion Matrix:
 [[2885   16]
 [  74   25]]
Accuracy Score 0.97


#### KFoldCV

In [51]:
from sklearn.model_selection import KFold, cross_val_score
cross_val_score(lr,X,y).mean()

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9733333333333334

# **Wine Dataset**

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score

In [54]:
wine = pd.read_csv("wine.csv")
wine.head()

,Class,Alcohol,Malic,Ash,Alcalinity,Magnesium,Phenols,Flavanoids,Nonflavanoid,Proanthocyanins,Intensity,Hue,OD280,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [56]:
y = wine["Class"]
X = wine.drop("Class", axis = 1)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [58]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [59]:
y_pred = lr.predict(X_test)

In [61]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAccuracy Score", accuracy_score(y_test, y_pred))

Confusion Matrix:
 [[20  1  0]
 [ 2 17  1]
 [ 0  1 12]]

Accuracy Score 0.9074074074074074
